<h2>An introduction to using python to access Madrigal data (madrigalWeb)</h2>

#### This notebook available at http://www.haystack.mit.edu/~brideout/Madrigal_python_demo.ipynb

To download the latest Python Madrigal API, go to http://cedar.openmadrigal.org/madrigalDownload/, or use pip:

<pre>pip install madrigalWeb</pre>

This module should work with either python 2 or python 3.

Then feel free to follow along with this talk!

Url of  Madrigal CEDAR database: http://cedar.openmadrigal.org

Online resources: <a href="http://cedar.openmadrigal.org/docs/name/rt_python.html">Tutorial</a> and <a href="http://http://cedar.openmadrigal.org/docs/name/rr_python.html">Reference</a>

Any questions: Email me (Bill Rideout) at <a href="mailto:brideout@mit.edu">brideout@haystack.mit.edu</a>

In [1]:
import madrigalWeb.madrigalWeb

The next command connects to one particular Madrigal site, and creates an object we will use for the rest of the session.

In [2]:
madDB = madrigalWeb.madrigalWeb.MadrigalData('http://cedar.openmadrigal.org')

The top level of metadata at any site is the instruments it has available. This next method lists the available instruments.

In [3]:
instList = madDB.getAllInstruments()

In [4]:
print(instList[7])

name: Millstone Hill IS Radar
code: 30
mnemonic: mlh
latitude: 42.619
longitude: 288.51
altitude: 0.146
category: Incoherent Scatter Radars



Note that the code associated with the Millstone Hill IS Radar is 30 (not related to its position in the list, which may change).  Also refered to as the kinst.  This will be used throughout the rest of the calls to refer to this instrument.

The next call gets all the experiments with that instrument over the date range of January 2016:

In [5]:
exps = madDB.getExperiments(30, 2016,1,1,0,0,0,2016,2,1,0,0,0)

In [6]:
print(exps[0])

id: 100174066
realUrl: http://cedar.openmadrigal.org/showExperiment/?experiment_list=100174066
url: http://cedar.openmadrigal.org/madtoc/experiments/2016/mlh/07jan16
name: Repeatable Calibration
siteid: 10
sitename: CEDAR
instcode: 30
instname: Millstone Hill IS Radar
startyear: 2016
startmonth: 1
startday: 7
starthour: 15
startmin: 37
startsec: 48
endyear: 2016
endmonth: 1
endday: 7
endhour: 20
endmin: 55
endsec: 31
isLocal: True
madrigalUrl: http://cedar.openmadrigal.org/
PI: Phil Erickson
PIEmail: perickson@haystack.mit.edu
uttimestamp: 1566865148
access: 2
Madrigal version: 3.0



In [7]:
len(exps)

7

The next call gets all the files in that experiment.  An experiment may contain different types of data, as will be described in the kind of data description (kindatdesc):

In [8]:
files = madDB.getExperimentFiles(exps[0].id)

In [9]:
print(files[0])

name: /opt/madrigal3/experiments/2016/mlh/07jan16/mlh160107g.007.hdf5
kindat: 3410
kindatdesc: Combined basic parameters file - all antennas and modes
category: 1
status: Final
permission: 0
expId: 100174066
doi: https://w3id.org/cedar?experiment_list=experiments/2016/mlh/07jan16&file_list=mlh160107g.007.hdf5



As a simple example, let's just download this data as is.  You could also filter the data, add derived parameters, and select the parameters you want using other calls.  

Because Madrigal is not login controlled, logging is done by you passing in unverified identification information - your name, email, and affiliation.  Below I set up my personal info - <b>please change this to yours when you use this API!</b>

In [10]:
user_fullname = 'Student Example' 
user_email = 'isr.summer.school@gmail.com' 
user_affiliation= 'ISR Summer School 2020'

In [11]:
madDB.downloadFile(files[0].name, '/tmp/test.hdf5', user_fullname, user_email, user_affiliation, format='hdf5')

Now let's say I wanted to know what parameters were in this file.  You can get that from the API also:

In [12]:
parms = madDB.getExperimentFileParameters(files[0].name)

In [13]:
print(parms[25])

mnemonic: DVTX
description: Error in Transmitter phase velocity
isError: 1
units: m/s
isMeasured: 1
category: I. S. Radar Operation Parameter
isSure: 1
isAddIncrement: -1



In the next command we are going to download data from that file with the parameters we select, and the filters we want to remove unwanted data.  A list of all valid CEDAR Madrigal parameters can be found at http://madrigal3.haystack.mit.edu/parameterMetadata/.  A description of how to write filter strings are at http://madrigal3.haystack.mit.edu/filterMetadata/ .

In this example, we will choose the parameters YEAR,MONTH,DAY,HOUR,MIN,SEC,AZM,ELM,RANGE,GDALT,GLON,GDALT,TI,DTI. The filters will limit gdalt to between 200 and 300, and we will only look at data where DTI < 10 degrees. Our filter string is then "filter=gdalt,200,300 filter=dti,,10" The blank in dti means no lower limit.

In [14]:
filterStr="filter=gdalt,200,300 filter=dti,,10"
parms = "YEAR,MONTH,DAY,HOUR,MIN,SEC,AZM,ELM,RANGE,GDALT,GLON,GDALT,TI,DTI"
result = madDB.isprint(files[0].name, parms, filterStr, user_fullname, user_email, user_affiliation)

In [15]:
lines = result.split('\n')
for line in lines[:5]:
    print(line)

     2016         1         7        15        39        48    178.00      88.00     231.56     231.54     -71.49      929.0        7.0  
     2016         1         7        15        39        48    178.00      88.00     249.55     249.52     -71.49      907.0        4.0  
     2016         1         7        15        39        48    178.00      88.00     267.53     267.49     -71.49      905.0        5.0  
     2016         1         7        15        39        48    178.00      88.00     285.52     285.47     -71.49      897.0        5.0  
     2016         1         7        15        44         3    178.13      88.02     267.30     267.26     -71.49      889.0        2.0  
